# CONTRATO DE INTEGRACIÓN DS-BACKEND
## ChurnInsight - Hackathon NoCountry 2025 - H12-25-L-Equipo 43

---

### TECNOLOGÍA
- **Framework DS:** FastAPI
- **Puerto:** 8000 (configurable)
- **Endpoint:** POST `/predict`

---

### REQUEST BODY (lo que Backend envía a DS)
```json
{
  "antiguedad": 18,
  "plan": "Premium",
  "metodo_pago": "Tarjeta de Credito",
  "facturas_impagas": 1,
  "frecuencia_uso": 22,
  "tickets_soporte": 3,
  "tipo_contrato": "Mensual",
  "cambios_plan": 1,
  "canal_adquisicion": "Web"
}
```

**Todos los campos son obligatorios.**

**Descripción de campos:**
- `antiguedad` (int): Meses como cliente (0-120)
- `plan` (string): Tipo de plan ["basico", "estandar", "premium"]
- `metodo_pago` (string): Método de pago ["transferencia_bancaria", "tarjeta_credito", "tarjeta_debito", "efectivo"]
- `facturas_impagas` (int): Cantidad de facturas sin pagar (0-10)
- `frecuencia_uso` (int): Días activos por mes (0-30)
- `tickets_soporte` (int): Cantidad de tickets de soporte-ayuda abiertos (0-50)
- `tipo_contrato` (string): Compromiso temporal ["mensual", "anual"]
- `cambios_plan` (int): Cambios de plan en últimos 6 meses (0-5)
- `canal_adquisicion` (string): Canal de llegada ["web", "referido", "redes_sociales", "call_center"]
---

### RESPONSE BODY (200 OK) (lo que DS devuelve a Backend)
```json
{
  "prediccion": "No cancelará",
  "probabilidad_churn": 0.23,
  "top_features": [
    {
      "feature": "Frecuencia_Uso",
      "valor": 22,
      "impacto": "Reduce el riesgo"
    },
    {
      "feature": "Antiguedad",
      "valor": 18,
      "impacto": "Reduce el riesgo"
    },
    {
      "feature": "Facturas_Impagas",
      "valor": 1,
      "impacto": "Aumenta el riesgo"
    }
  ],
  "modelo_version": "v1.0"
}
```

**Descripción:**
- `prediccion` (string): Texto interpretable ["Va a cancelar", "No cancelará"]
- `probabilidad_churn` (float): Probabilidad de cancelación (0.0-1.0)
 - `top_features` (array): **3 variables más influyentes** según el modelo entrenado
  - `feature`: Nombre de la variable
  - `valor`: Valor que tiene el cliente
  - `impacto`: Si aumenta o reduce el riesgo
- `modelo_version` (string): Versión del modelo (para logs/auditoría)

**NOTA IMPORTANTE:** Los nombres exactos de las top 3 features se definirán después del entrenamiento del modelo. Backend debe estar preparado para recibir cualquier combinación de las 9 features de entrada.

---

### ERRORES (400 Bad Request)
```json
{
  "error": "Validación fallida",
  "detalles": [
    {
      "campo": "plan",
      "mensaje": "Valor inválido. Debe ser: Básico, Estandar o Premium"
    }
  ]
}
```

---

### INTEGRACIÓN

**Arquitectura:**
```
[Frontend] → [Backend Spring Boot :8080] → [Microservicio DS FastAPI :8000] → [Modelo ML]
```

**Flujo:**
1. Frontend envía formulario a Backend
2. Backend valida y hace POST a `http://localhost:8000/predict`
3. DS (FastAPI) carga modelo, predice y devuelve JSON
4. Backend enriquece response (agrega timestamp, guarda en BD) y responde a Frontend

**Responsabilidades:**
- **Backend:** Validación de entrada, persistencia, logs, timestamp
- **DS:** Solo predicción ML (sin BD, sin timestamp, sin validaciones de negocio)

---

### CASOS DE PRUEBA

**1. Cliente de alto riesgo (prob > 0.7):**
```json
{
  "antiguedad": 3,
  "plan": "Básico",
  "metodo_pago": "Efectivo",
  "facturas_impagas": 2,
  "frecuencia_uso": 4,
  "tickets_soporte": 8,
  "tipo_contrato": "Mensual",
  "cambios_plan": 2,
  "canal_adquisicion": "Call Center"
}
```

**2. Cliente de bajo riesgo (prob < 0.3):**
```json
{
  "antiguedad": 48,
  "plan": "Premium",
  "metodo_pago": "Tarjeta de Debito",
  "facturas_impagas": 0,
  "frecuencia_uso": 28,
  "tickets_soporte": 0,
  "tipo_contrato": "Anual",
  "cambios_plan": 0,
  "canal_adquisicion": "Referido"
}
```

**3. Cliente de riesgo medio (prob 0.3-0.7):**
```json
{
  "antiguedad": 18,
  "plan": "Estandar",
  "metodo_pago": "Tarjeta de Credito",
  "facturas_impagas": 1,
  "frecuencia_uso": 15,
  "tickets_soporte": 3,
  "tipo_contrato": "Mensual",
  "cambios_plan": 1,
  "canal_adquisicion": "Web"
}
```

---

### EJEMPLO DE USO (cURL)
```bash
curl -X POST http://localhost:8000/predict \
  -H "Content-Type: application/json" \
  -d '{
    "antiguedad": 18,
    "plan": "Premium",
    "metodo_pago": "Tarjeta de Credito",
    "facturas_impagas": 1,
    "frecuencia_uso": 22,
    "tickets_soporte": 3,
    "tipo_contrato": "Mensual",
    "cambios_plan": 1,
    "canal_adquisicion": "Web"
  }'
```

---

### CONSIDERACIONES TÉCNICAS

**¿Por qué FastAPI?**
- Validación automática con Pydantic
- Documentación Swagger en `/docs` (automática)
- Más rápido que Flask
- Type hints nativos (menos bugs)

**¿Por qué puerto 8000?**
- Convención FastAPI (configurable)
- Backend en 8080, DS en 8000 (separación clara)

**¿Timestamp y confianza?**
- NO incluidos en response DS
- Backend puede agregar `timestamp` al guardar en BD
- Frontend puede calcular "confianza" de la `probabilidad_churn`:
  - `if prob < 0.3: "🟢 Riesgo Bajo"`
  - `if 0.3 <= prob <= 0.7: "🟡 Riesgo Medio"`
  - `if prob > 0.7: "🔴 Riesgo Alto"`